<a href="https://colab.research.google.com/github/vnprr/CatsVsDogs/blob/main/CatsVsDogs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Prepare data and disc

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# imports
import glob
import tensorflow as tf
from tensorflow.keras import layers, models
from tqdm.notebook import tqdm
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

In [ ]:
!mkdir -p ~/.kaggle
!cp /content/drive/MyDrive/Programowanie/CatsVsDogs/KaggleToken/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c dogs-vs-cats-redux-kernels-edition

In [ ]:
!unzip -q dogs-vs-cats-redux-kernels-edition.zip

!unzip -q train.zip

!rm test.zip

In [ ]:
N_PER_SAMPLE = 1000
path_cats = glob.glob("train/cat.*.jpg")[: N_PER_SAMPLE]
path_dogs = glob.glob("train/dog.*.jpg")[: N_PER_SAMPLE]

In [ ]:
# #test images

# import cv2
# cat = cv2.imread(path_cats[1])

# from google.colab.patches import cv2_imshow

# for i in range(20):
#   cat = cv2.imread(path_cats[i])
#   cv2_imshow(cat)


In [ ]:
# Image statistics collection, calculation of average height and width.

heights = []
widths = []

for path in path_cats + path_dogs:
    # load img
    img = cv2.imread(path)

    # image size
    height, width, channels = img.shape
    heights.append(height)
    widths.append(width)

IMG_HEIGHT = int(np.round(np.mean(heights)))
IMG_WIDTH = int(np.round(np.mean(widths)))

print(IMG_HEIGHT, IMG_WIDTH)

## Image Generator

In [ ]:
%pwd
!ls train/cats | wc -l
!ls train/dogs | wc -l

In [ ]:
# manage files

import os
import shutil

# path
train_folder = "./train"

# Create subfolders for cats and dogs if they don't exist
os.makedirs(os.path.join(train_folder, "cats"), exist_ok=True)
os.makedirs(os.path.join(train_folder, "dogs"), exist_ok=True)

# Iterate through all files in the "train" folder
for filename in os.listdir(train_folder):
    if filename.endswith(".jpg") or filename.endswith(".png"):  # Adjust file extensions if needed
        file_path = os.path.join(train_folder, filename)

        # Move cat images to the "cats" subfolder
        if "cat" in filename.lower():
            shutil.move(file_path, os.path.join(train_folder, "cats"))

        # Move dog images to the "dogs" subfolder
        if "dog" in filename.lower():
            shutil.move(file_path, os.path.join(train_folder, "dogs"))

In [ ]:
# take 2500 images for test

# path
test_folder = "./test"

# Create subfolders for cats and dogs if they don't exist
os.makedirs(os.path.join(test_folder, "cats"), exist_ok=True)
os.makedirs(os.path.join(test_folder, "dogs"), exist_ok=True)

# Iterate through all files in the "train" folder
for subdir in os.listdir(train_folder):
    for filename in os.listdir(os.path.join(train_folder, subdir)):
      if filename.endswith(".jpg") or filename.endswith(".png"):  # Adjust file extensions if needed
          if filename.split('.')[1] in [str(i) for i in range(10000, 12500)]:
            file_path = os.path.join(train_folder, subdir, filename)
            # Move cat images to the "cats" subfolder
            if "cat" in filename.lower():
                out_path = os.path.join(test_folder, 'cats', filename)
                shutil.move(file_path, out_path)
            # Move dog images to the "dogs" subfolder
            if "dog" in filename.lower():
                out_path = os.path.join(test_folder, 'dogs', filename)
                shutil.move(file_path, out_path)




In [ ]:
!ls test/cats | wc -l
!ls test/dogs | wc -l
!ls train/cats | wc -l
!ls train/dogs | wc -l

In [ ]:
import cv2
import tensorflow as tf # Import tensorflow

def pf(image):
  image = tf.image.resize_with_pad(image, IMG_WIDTH, IMG_HEIGHT) # Resize and pad images to ensure consistent dimensions
  return image

img_gen = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=pf,
    validation_split=0.2
)

#train dataset
BATCH_SIZE = 64
train_ds = img_gen.flow_from_directory(
    "./train",
    target_size=[IMG_WIDTH, IMG_HEIGHT],
    batch_size=BATCH_SIZE,
    shuffle=True,
    class_mode='binary', # Change class_mode to binary
    color_mode='grayscale',
    subset='training'
)

#val dataset
val_ds = img_gen.flow_from_directory(
    "./train",
    target_size=[IMG_WIDTH, IMG_HEIGHT],
    batch_size=BATCH_SIZE,
    shuffle=True,
    class_mode='binary', # Change class_mode to binary
    color_mode='grayscale',
    subset='validation'
)


## Train

In [ ]:
import tensorflow as tf
from tensorflow.keras import models, layers

# build model by using input
model = models.Sequential([
    layers.Input(shape=(IMG_WIDTH, IMG_HEIGHT, 1)),  # input shape
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.2),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.2),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.2),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.2),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.2),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

# compile model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])



In [ ]:
early = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=5,
    restore_best_weights=True
)

#model 2
history = model.fit(train_ds, validation_data=val_ds, epochs=50, batch_size=1024, callbacks=[early])  # pobawić się batch size żeby było szybciej
# history = model.fit(train_ds, validation_data=val_ds, epochs=10, batch_size=2048)  # pobawić się batch size żeby było szybciej

### Evaluate

In [ ]:
import matplotlib.pyplot as plt

# Get the training history data
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

# Create the accuracy plot
plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

# Create the loss plot
plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Loss')
plt.ylim([0,max(plt.ylim())])
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.show()

In [ ]:
# save model and history
import pickle

model.save('/content/drive/MyDrive/Programowanie/CatsVsDogs/Modele/cat_dog_classifier1_25e.keras')
with open('/content/drive/MyDrive/Programowanie/CatsVsDogs/Modele/cat_dog_classifier1_25e_history.pkl', 'wb') as f:
    pickle.dump(history.history, f)

## Test

In [ ]:
def pf(image):
  image = tf.image.resize_with_pad(image, IMG_WIDTH, IMG_HEIGHT) # Resize and pad images to ensure consistent dimensions
  return image

img_gen_test = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=pf,
)

#train dataset
BATCH_SIZE = 64
test_ds = img_gen_test.flow_from_directory(
    "./test",
    target_size=[IMG_WIDTH, IMG_HEIGHT],
    batch_size=BATCH_SIZE,
    shuffle=True,
    class_mode='binary', # Change class_mode to binary
    color_mode='grayscale',
)


In [ ]:
# test model
results = model.evaluate(test_ds)
print(f"Test Loss: {results[0]}, Test Accuracy: {results[1]}")